In [46]:
import pandas as pd
import numpy as np
from nltk import ngrams
from emoji import UNICODE_EMOJI
import twokenize
import string
from itertools import groupby

datafile = "datasets/train/SemEval2018-T3-train-taskA.txt"
datafile2 = "datasets/train/SemEval2018-T3-train-taskA_emoji_ironyHashtags.txt"
trainingdata = pd.read_csv(datafile2, sep = "\t", header=0)

In [47]:
trainingdata = trainingdata[['Label','Tweet text']]


In [48]:
type(trainingdata['Tweet text'])
tweets = trainingdata['Tweet text']

In [49]:
allbags = []
for i in range(len(trainingdata)):
    bagofwords = twokenize.simpleTokenize(tweets.iloc[i])
    allbags.append(bagofwords)
trainingdata["Bag"] = allbags



In [50]:
allbigrams = []
for i in range(len(trainingdata)):
    bigram = ngrams(allbags[i],2)
    allbigrams.append(bigram)
bigramlist=[]
for i in range(len(trainingdata)):
    value = list(allbigrams[i])
    bigramlist.append(value)
trainingdata["Bigram"] = bigramlist


In [51]:
n=3
chartrigram =[]
for i in range(len(trainingdata)):
    holdgram = []
    for j in range(len(allbags[i])):
        holdgram.extend([allbags[i][j][k:k+n] for k in range(len(allbags[i][j])-n+1)])
    chartrigram.append(holdgram)
n=4
charfourgram=[]
for i in range(len(trainingdata)):
    holdgram = []
    for j in range(len(allbags[i])):
        holdgram.extend([allbags[i][j][k:k+n] for k in range(len(allbags[i][j])-n+1)])
    charfourgram.append(holdgram)
trainingdata["CharTrigram"] = chartrigram
trainingdata["CharFourgram"] = charfourgram

In [52]:

trainingdata.iloc[i,2][2]
len(trainingdata.iloc[i,2])


23

In [53]:
# count = 0
# puncfloodfeature=[]
# for i in range(len(trainingdata)):
#     puncflood = 0
#     for j in range(len(trainingdata.iloc[i,2])):
#         if (trainingdata.iloc[i,2][j] == "..." or trainingdata.iloc[i,2][j] == ".."):
#          puncflood += 1
#     puncfloodfeature.append(puncflood)     
# trainingdata["PunctuationFlood"] = puncfloodfeature

puncfloodfeature =[]
for i in range(len(trainingdata)):
    counter = 0
    s = trainingdata.iloc[i,1]
    t = [[k,len(list(g))] for k, g in groupby(s)]
    for element in t:
        char = element[0]
        count = element[1]
        if char in string.punctuation and count > 1:
            counter += 1
    puncfloodfeature.append(counter)
trainingdata["PunctuationFlood"] = puncfloodfeature

In [54]:
# strs="asssssassin"
# a = Counter(strs)
# b=[[r, len(list(g))] for r, g in groupby(strs)]
# max(b)[1]
charfloodfeature = []
for i in range(len(trainingdata)):
    charflood = 0
    for j in range(len(trainingdata.iloc[i,2])):
        b = [[r, len(list(g))] for r, g in groupby(trainingdata.iloc[i,2][j])]
        if (max(b)[1]>2):
            charflood += 1
    charfloodfeature.append(charflood)   
charfloodfeature = [max(0,x1 - x2) for (x1, x2) in zip(charfloodfeature, puncfloodfeature)]
trainingdata["CharFlood"] = charfloodfeature

In [55]:
# charfloodfeature.index(1)
# trainingdata.iloc[60,2] 

In [56]:
str = "Hey"
capitalizefeature=[]
for i in range(len(trainingdata)):
    capfeat = 0
    for j in range(len(trainingdata.iloc[i,2])):
        if (trainingdata.iloc[i,2][j][0].isupper()):
            capfeat += 1
    capitalizefeature.append(capfeat)  
trainingdata["CapitalizedCount"] = capitalizefeature

In [57]:
hashtagfeature=[]
for i in range(len(trainingdata)):
    hashtags = 0
    for j in range(len(trainingdata.iloc[i,2])):
        if (trainingdata.iloc[i,2][j][0] == "#"):
            hashtags += 1
    hashtagfeature.append(hashtags)  
trainingdata["HashtagCount"] = hashtagfeature


In [58]:
hashtagtowordfeature=[]
for i in range(len(trainingdata)):
    if (hashtagfeature[i] != 0):
        hashtagratio = hashtagfeature[i]/len(trainingdata.iloc[i,2])
    else:
        hashtagratio = 0
    hashtagtowordfeature.append(hashtagratio)
trainingdata["Hashtag2WordRatio"] = hashtagtowordfeature


In [59]:
tweetlen=[]
for i in range(len(trainingdata)):
    tweetlen.append(len(trainingdata.iloc[i,2]))
trainingdata["TweetLength"] = tweetlen


In [60]:
def is_emoji(s):
    count = 0
    for emoji in UNICODE_EMOJI:
        count += s.count(emoji)
        if count > 1:
            return False
    return bool(count)
emojifeature=[]
for i in range(len(trainingdata)):
    emojicount = 0
    for j in range(len(trainingdata.iloc[i,2])):
        if (is_emoji(trainingdata.iloc[i,2][j])):
            emojicount += 1;
    emojifeature.append(emojicount)
trainingdata["EmojiCount"] = emojifeature


In [61]:
trainingdata

,Label,Tweet text,Bag,Bigram,CharTrigram,CharFourgram,PunctuationFlood,CharFlood,CapitalizedCount,HashtagCount,Hashtag2WordRatio,TweetLength,EmojiCount
0,1,Sweet United Nations video. Just in time for C...,"[Sweet, United, Nations, video, ., Just, in, t...","[(Sweet, United), (United, Nations), (Nations,...","[Swe, wee, eet, Uni, nit, ite, ted, Nat, ati, ...","[Swee, weet, Unit, nite, ited, Nati, atio, tio...",1,0,5,3,0.200000,15,0
1,1,@mrdahl87 We are rumored to have talked to Erv...,"[@mrdahl87, We, are, rumored, to, have, talked...","[(@mrdahl87, We), (We, are), (are, rumored), (...","[@mr, mrd, rda, dah, ahl, hl8, l87, are, rum, ...","[@mrd, mrda, rdah, dahl, ahl8, hl87, rumo, umo...",2,0,5,1,0.041667,24,0
2,1,Hey there! Nice to see you Minnesota/ND Winter...,"[Hey, there, !, Nice, to, see, you, Minnesota/...","[(Hey, there), (there, !), (!, Nice), (Nice, t...","[Hey, the, her, ere, Nic, ice, see, you, Min, ...","[ther, here, Nice, Minn, inne, nnes, neso, eso...",0,0,5,1,0.090909,11,0
3,0,3 episodes left I'm dying over here,"[3, episodes, left, I'm, dying, over, here]","[(3, episodes), (episodes, left), (left, I'm),...","[epi, pis, iso, sod, ode, des, lef, eft, I'm, ...","[epis, piso, isod, sode, odes, left, dyin, yin...",0,0,1,0,0.000000,7,0
4,1,I can't breathe! was chosen as the most notabl...,"[I, can't, breathe, !, was, chosen, as, the, m...","[(I, can't), (can't, breathe), (breathe, !), (...","[can, an', n't, bre, rea, eat, ath, the, was, ...","[can', an't, brea, reat, eath, athe, chos, hos...",0,0,3,1,0.040000,25,0
5,0,You're never too old for Footie Pajamas. http:...,"[You're, never, too, old, for, Footie, Pajamas...","[(You're, never), (never, too), (too, old), (o...","[You, ou', u'r, 're, nev, eve, ver, too, old, ...","[You', ou'r, u're, neve, ever, Foot, ooti, oti...",1,0,3,0,0.000000,9,0
6,1,Nothing makes me happier then getting on the h...,"[Nothing, makes, me, happier, then, getting, o...","[(Nothing, makes), (makes, me), (me, happier),...","[Not, oth, thi, hin, ing, mak, ake, kes, hap, ...","[Noth, othi, thin, hing, make, akes, happ, app...",1,0,2,1,0.047619,21,0
7,0,4:30 an opening my first beer now gonna be a l...,"[4:30, an, opening, my, first, beer, now, gonn...","[(4:30, an), (an, opening), (opening, my), (my...","[4:3, :30, ope, pen, eni, nin, ing, fir, irs, ...","[4:30, open, peni, enin, ning, firs, irst, bee...",0,0,0,0,0.000000,12,0
8,0,@Adam_Klug do you think you would support a gu...,"[@Adam_Klug, do, you, think, you, would, suppo...","[(@Adam_Klug, do), (do, you), (you, think), (t...","[@Ad, Ada, dam, am_, m_K, _Kl, Klu, lug, you, ...","[@Ada, Adam, dam_, am_K, m_Kl, _Klu, Klug, thi...",0,0,1,0,0.000000,20,0
9,0,@samcguigan544 You are not allowed to open tha...,"[@samcguigan544, You, are, not, allowed, to, o...","[(@samcguigan544, You), (You, are), (are, not)...","[@sa, sam, amc, mcg, cgu, gui, uig, iga, gan, ...","[@sam, samc, amcg, mcgu, cgui, guig, uiga, iga...",0,0,2,0,0.000000,12,0
